[Markdown styles](https://towardsdatascience.com/enrich-your-jupyter-notebook-with-these-tips-55c8ead25255)

[Hydra overview](https://towardsdatascience.com/stop-hard-coding-in-a-data-science-project-use-config-files-instead-479ac8ffc76f)

<u>credits</u>: [Will Koehrsen](https://medium.com/p/4d028e6f0526)

<h1 style="text-align:center"><u>Book Recommendation System</u></h1>

<h2>Introduction</h2>

The main purpose of this work is to learn <font color='orange'>**embeddings**</font>. Embeddings can be considered as a representation of a document in a space (called **embedding space**) where similar documents should be very close to each other. Therefore to guide our comprehension of what are embeddings, we will build a recommendation system based on books in wikipedia. The principle is simple: "<u>`books with Wikipedia pages that link to similar Wikipedia pages are similar to each other`</u>". Ex: Book1--> link2; Book3--> link2 => Book1 and Book3 are similar. So, to exhibit this similarity, we will use neural network entity embeddings, mapping each book and each Wikipedia link (Wikilink) to a 50-number vector.

Embeddings have demonstrated some advantages in representing categorical variables: 1- They help in reducing the original space where documents are expressed. In our case for example, if we were using *one-hot-encoding* method to representing books, we would have 3.7e+06 length vector for each book. 2- They preserve similarity in terms of semantic meaning of each document (the books in our case) that is not the case of *one-hot-encoding* where semantic meaning of document is not preserved. By training a neural network to learn entity embeddings, we not only get a reduced dimension representation of the books, we also get a representation that keeps similar books (in terms of what they talk about) closer to each other. Basic approach for a recommendation system is to find the closest books for any book in other to recommend to a user books that may have same content like those he/she already read. Thanks to [this notebook](1-Downloading%20and%20Parsing%20Wikipedia%20Articles.ipynb), we have access to every single book article on Wikipedia, which will let us create an effective recommendation system.

<h2>Approach</h2>

To create entity embeddings, we need to <u>build an embedding neural network</u> and train it <u>on a supervised machine learning task</u> that will <u>result in similar books (and similar links) having closer representations</u> in embedding space. The parameters of the neural network - the weights - are the embeddings, and so during training, these numbers are adjusted to minimize the loss on the prediction problem. In other words, the network tries to accurately complete the task by changing the representations of the books and the links.

Once we have the embeddings for the books and the links, we can find the most similar book to a given book by computing the distance between the embedded vector for that book and all the other book embeddings. <u>We'll use the cosine distance</u> which measures the angle between two vectors as a measure of similarity (another valid option is the Euclidean distance). We can also do the same with the links, finding the most similar page to a given page. (I use links and wikilinks interchangeably in this notebook). The steps we will follow are:

1. Load in data and clean
2. Prepare data for supervised machine learning task
3. Build the entity embedding neural network
4. Train the neural network on prediction task
5. Extract embeddings and find most similar books and wikilinks
6. Visualize the embeddings using dimension reduction techniques

*Question*: But what consists the Supervised Machine Learning Task ? 🤔<br/>
*Answer*: Find the model that correctly <u>Maps Books to Links</u> 😀

For our machine learning task, we'll set up the problem as <u>identifying</u> <font color='green'>whether</font> <u>or</u> <font color='red'>not</font> <u>a particular link was present in a book article</u>. The training examples will consist of (book, link) pairs, with some pairs true examples - actually in the data - and others negative examples - do not occur in the data. It will be the network's job to adjust the entity embeddings of the books and the links in order to accurately make the classification. 

|                      Books                          | Links |  Link present/absent ?  |
|:---------------------------------------------------:|:---------:|:------:|
| Book4|   wikilink1   |  <font color='green'>Present</font>  |
| Book4|   wikilink7   |  <font color='red'>Absent<font>   |
| Book4|   wikilink8   |  <font color='green'>Present</font>  |
| Book2|   wikilink7   |  <font color='red'>Absent<font>   |
| Book2|   wikilink1   |  <font color='green'>Present</font>  |
| Book5|   wikilink3   |  <font color='red'>Absent<font>   |
| Book5|   wikilink5   |  <font color='red'>Absent<font>   |
| Book5|   wikilink1   |  <font color='green'>Present</font>  |
| Book1|   wikilink5   |  <font color='green'>Present</font>  |

<u>**Keep in mind**</u>: Although we are training for a supervised machine learning task, our end objective is not to make accurate predictions on new data, but learn the best entitiy embeddings, so we do not use a validation or testing set. We use the prediction problem as a means to an end rather than the final outcome.

<h2>Neural Network Embeddings for better representing large sequence of texts</h2>


Neural Network embeddings have proven to be very powerful concepts both for modeling language and for representing categorical variables.  For example, the [Word2Vec](https://www.tensorflow.org/tutorials/text/word2vec?hl=fr) word embeddings by Google, map a word to a vector based on training a neural network on millions of words. These embeddings can be used in any supervised model because they are just numerical representations of categorical variables. Much as we one-hot-encode categorical variables to use them in a random forest for a supervised task, we can also use entity embeddings to include categorical variables in a model. The embeddings are also useful because we can find entities that are close to one another in embedding space which might - as in a book recommendation system - allow us to find the most similar categories among tens of thousands of choices.

We can also use the Entity Embeddings to visualize words or categorical variables, such as creating a map of all books on Wikipedia. The entity embeddings typically are still high-dimensional - we'll use 50 numbers for each entity - so we need to use a dimension reduction technique such as [TSNE](https://distill.pub/2016/misread-tsne/) or [UMAP](https://umap-learn.readthedocs.io/en/latest/how_umap_works.html) to visualize the embeddings in lower dimensions. (These are both manifold embedding methods so in effect we will embed the embeddings for visualization!) We'll take a look at doing this at the end of the notebook and later will upload the embeddings into a application custom-built for this purpose ([projector.tensorflow.org](https://projector.tensorflow.org/)). Entity embeddings are becoming more widespread thanks to the ease of development of neural networks in Keras and are a useful approach when we want to represent categorical variables with vectors that place similar categories close to one another. Other approaches for encoding categorical variables do not represent similar entities as being closer to one another, and entity embedding is a learning-based method for this important task.

Overall, this project is a great look at the potential for neural networks to create meaningful embeddings of high dimensional data and a practical application of deep learning. The code itself is relatively simple, and the Keras library makes developing deep learning models enjoyable!

The code here is adapted from the excellent [Deep Learning Cookbook](https://www.amazon.com/Deep-Learning-Cookbook-Practical-Recipes/dp/149199584X), the notebooks for which can be found on [this GitHub repository]((https://github.com/DOsinga/deep_learning_cookbook/tree/master)). Check out this book for practical applications of deep learning and great projects!

<h3>1. Load in data and clean</h3>

> Read in Data

The data is stored as json with line for every book. This data contains every single book article on Wikipedia which was parsed in the [Downloading and Parsing Wikipedia Data](1-Downloading%20and%20Parsing%20Wikipedia%20Articles.ipynb) Notebook.

In [1]:
from IPython.core.interactiveshell import InteractiveShell

# Set shell to show all lines of output
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import json

books = []

with open('./data/found_books_filtered.ndjson', 'r') as file:
    # Append each line to the books
    books = [json.loads(row) for row in file]

# Remove non-book articles
books_with_wikipedia = [book for book in books if 'Wikipedia:' in book[0]]
books = [book for book in books if 'Wikipedia:' not in book[0]]
print(f'Found {len(books)} books.')

Found 37020 books.


> Data Cleaning

There are a few articles that were caught which are clearly not books (feel free to check out these articles yourself).

In [11]:
[book[0] for book in books_with_wikipedia][:5]

['Wikipedia:Wikipedia Signpost/2014-06-25/Recent research',
 'Wikipedia:New pages patrol/Unpatrolled articles/December 2010',
 'Wikipedia:Templates for discussion/Log/2012 September 23',
 'Wikipedia:Articles for creation/Redirects/2012-10',
 'Wikipedia:Templates for discussion/Log/2012 October 4']

Each legitimate book contains the title, the information from the Infobox book template, the internal wikipedia links, the external links, the date of last edit, and the number of characters in the article (a rough estimate of the length of the article).

<img src="static\book_template.PNG" width=20% align ='center'>

In [16]:
n = 7
print(f"{books[n][0]};\n\n {books[n][1]};\n\n {books[n][2][:5]};\n\n {books[n][3][:5]};\n\n {books[n][3][:5]};\n\n {books[n][4]};\n\n {books[n][5]}")

Wild Boy (novel);

 {'1': '< !-- See Wikipedia:WikiProject Novels or Wikipedia:WikiProject Books -- >', 'name': 'Wild Boy', 'image': 'File:WildBoy.jpg', 'caption': 'First edition', 'author': 'Jill Dawson', 'cover_artist': 'Matt Harris (photographs)', 'country': 'United Kingdom', 'language': 'English', 'publisher': 'Sceptre', 'pub_date': '2003', 'media_type': 'Print', 'pages': '291', 'isbn': '0-340-82296-1'};

 ['Wikipedia:WikiProject Novels', 'Wikipedia:WikiProject Books', 'Jill Dawson', 'Sceptre (imprint)', 'Jill Dawson'];

 ['http://mattharrisphotography.co.uk/', 'https://www.theguardian.com/books/2003/sep/21/fiction.features', 'https://www.theguardian.com/books/2003/sep/27/featuresreviews.guardianreview20', 'https://www.telegraph.co.uk/culture/books/3603046/Child-of-the-forest.html', 'http://jilldawson.co.uk/pages/wild-boy/'];

 ['http://mattharrisphotography.co.uk/', 'https://www.theguardian.com/books/2003/sep/21/fiction.features', 'https://www.theguardian.com/books/2003/sep/27/fea

We will only use the wikilinks, which are saved as the third element (index 2) for each book.

<h3>2. Prepare data for supervised machine learning task</h3>

> Map books to integers

First we want to create a mapping of book titles to integers. When we feed books into the embedding neural network, we will have to represent them as numbers, and this mapping will let us keep track of the books. We'll also create the reverse mapping, from integers back to the title.

In [17]:
book_index = {book[0]: idx for idx, book in enumerate(books)}
index_book = {idx: book for book, idx in book_index.items()}

book_index['Anna Karenina']
index_book[22494]

22494

'Anna Karenina'

> Exploring Wikilinks

Although it's not our main focus, we can do a little exploration. Let's find the number of unique Wikilinks and the most common ones. To create a single list from a list of lists, we can use the `itertools` chain method.

In [18]:
from itertools import chain

wikilinks = list(chain(*[book[2] for book in books]))
print(f"There are {len(set(wikilinks))} unique wikilinks.")

There are 311276 unique wikilinks.
